In [ ]:
import os
import re
import time
from dotenv import load_dotenv
from github import Github

load_dotenv()



token = os.getenv('GITHUB_TOKEN')
g = Github(token)

# Trabajar sobre el repositorio específico
repo = g.get_repo("AlejandroDiBattista/tup25-p4")

print(f"         Repositorio: {repo.name}")
print(f"         Descripción: {repo.description}")
print(f"           Estrellas: {repo.stargazers_count}")
print(f"               Forks: {repo.forks_count}")
print(f"  Lenguaje principal: {repo.language}")
print(f"Última actualización: {repo.updated_at}")

         Repositorio: tup25-p4
         Descripción: TUP 2025 - Programación 4
           Estrellas: 5
               Forks: 64
  Lenguaje principal: JavaScript
Última actualización: 2025-09-05 07:46:07+00:00


In [ ]:
def normalizar_titulo_pr(titulo, alumnos):
    if match:= re.search(r'(\d{5})[,\s-]+(.+)\s*', titulo):
        legajo = match.group(1)
        nombre_alumno = next((a['nombre'] for a in alumnos if a['legajo'] == legajo), "Desconocido")
        return f"TP{tp} - {legajo} - {nombre_alumno} - {match.group(2).title()}"
    else:
        return titulo

def normalizar_pr(tp=2):

    alumnos = leer()


    open_pulls = repo.get_pulls(state='open', sort='created', base='main')
    print(f"> Pull Requests pendientes:")
    for pr in open_pulls:
        pr = repo.get_pull(pr.number)  # Refrescar datos del PR
        nuevo = normalizar_titulo_pr(pr.title, alumnos)
        if pr.title != nuevo:
            pr.edit(title=nuevo)
            print(f"- #{pr.number}: {nuevo} (por {pr.user.login}) {"*" if pr.title != nuevo else ""}")
    print(".")

# normalizar_pr(tp=2)
print(normalizar_titulo_pr("TP2 - 61037", leer()))  # Ejemplo de uso

> Leyendo alumnos.md...


FileNotFoundError: [Errno 2] No such file or directory: 'alumnos.md'

In [3]:
# Función para hacer merge de PRs sin conflictos
def merge_prs_sin_conflictos():
    open_pulls = repo.get_pulls(state='open', sort='created', base='main')
    print("> Intentando merge de PRs sin conflictos:")
    for pr in open_pulls:
        pr = repo.get_pull(pr.number)  # Refrescar datos
        # Esperar hasta que mergeable no sea None
        intentos = 0
        max_intentos = 12  # Máximo 12 * 5 = 60 segundos
        while pr.mergeable is None and intentos < max_intentos:
            print(f"Esperando mergeability para PR #{pr.number}... ({intentos+1}/{max_intentos})")
            time.sleep(5)
            pr = repo.get_pull(pr.number)  # Refrescar
            intentos += 1

        if pr.mergeable is True:  # Solo si puede ser merged
            try:
                pr.merge()
                print(f"✓ PR #{pr.number} merged: {pr.title}")
            except Exception as e:
                print(f"✗ Error al mergear PR #{pr.number}: {e}")
        elif pr.mergeable is False:
            print(f"✗ PR #{pr.number} tiene conflictos: {pr.title}")
        else:
            print(f"✗ Tiempo agotado para PR #{pr.number}: {pr.title}")
    print(".")

# Ejecutar la función
# merge_prs_sin_conflictos()